# Multiple Linear Regression with sklearn - Exercise Solution

You are given a real estate dataset. 

Real estate is one of those examples that every regression course goes through as it is extremely easy to understand and there is a (almost always) certain causal relationship to be found.

The data is located in the file: 'real_estate_price_size_year.csv'. 

You are expected to create a multiple linear regression (similar to the one in the lecture), using the new data. 

Apart from that, please:
-  Display the intercept and coefficient(s)
-  Find the R-squared and Adjusted R-squared
-  Compare the R-squared and the Adjusted R-squared
-  Compare the R-squared of this regression and the simple linear regression where only 'size' was used
-  Using the model make a prediction about an apartment with size 750 sq.ft. from 2009
-  Find the univariate (or multivariate if you wish - see the article) p-values of the two variables. What can you say about them?
-  Create a summary table with your findings

In this exercise, the dependent variable is 'price', while the independent variables are 'size' and 'year'.

Good luck!

In [1]:
%load_ext lab_black

## Import the relevant libraries

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

from sklearn import linear_model

## Load the data

In [20]:
data = pd.read_csv("reference/S4_L48/real_estate_price_size_year.csv")
data.head()

,price,size,year
0,234314.144,643.09,2015
1,228581.528,656.22,2009
2,281626.336,487.29,2018
3,401255.608,1504.75,2015
4,458674.256,1275.46,2009


In [21]:
data.describe()

,price,size,year
count,100.000000,100.000000,100.000000
mean,292289.470160,853.024200,2012.600000
std,77051.727525,297.941951,4.729021
min,154282.128000,479.750000,2006.000000
25%,234280.148000,643.330000,2009.000000
50%,280590.716000,696.405000,2015.000000
75%,335723.696000,1029.322500,2018.000000
max,500681.128000,1842.510000,2018.000000


## Create the regression

### Declare the dependent and the independent variables

In [22]:
x = data[["size", "year"]]
y = data["price"]

### Regression

In [26]:
# Since the p-values are obtained through certain statistics, we need the 'stat' module from scipy.stats
import scipy.stats as stat

# Since we are using an object oriented language such as Python, we can simply define our own
# LinearRegression class (the same one from sklearn)
# By typing the code below we will ovewrite a part of the class with one that includes p-values
# Here's the full source code of the ORIGINAL class: https://github.com/scikit-learn/scikit-learn/blob/7b136e9/sklearn/linear_model/base.py#L362


class LinearRegression(linear_model.LinearRegression):
    """
    LinearRegression class after sklearn's, but calculate t-statistics
    and p-values for model coefficients (betas).
    Additional attributes available after .fit()
    are `t` and `p` which are of the shape (y.shape[1], X.shape[1])
    which is (n_features, n_coefs)
    This class sets the intercept to 0 by default, since usually we include it
    in X.
    """

    # nothing changes in __init__
    def __init__(self, fit_intercept=True, normalize=False, copy_X=True, n_jobs=1):
        self.fit_intercept = fit_intercept
        self.normalize = normalize
        self.copy_X = copy_X
        self.n_jobs = n_jobs

    def fit(self, X, y, n_jobs=1):
        self = super(LinearRegression, self).fit(X, y, n_jobs)

        # Calculate SSE (sum of squared errors)
        # and SE (standard error)
        sse = np.sum((self.predict(X) - y) ** 2, axis=0) / float(
            X.shape[0] - X.shape[1]
        )
        se = np.array([np.sqrt(np.diagonal(sse * np.linalg.inv(np.dot(X.T, X))))])

        # compute the t-statistic for each feature
        self.t = self.coef_ / se
        # find the p-value for each feature
        self.p = np.squeeze(
            2 * (1 - stat.t.cdf(np.abs(self.t), y.shape[0] - X.shape[1]))
        )
        return self

In [27]:
!cat /proc/cpuinfo

processor	: 0
BogoMIPS	: 52.00
Features	: fp asimd evtstrm aes pmull sha1 sha2 crc32
CPU implementer	: 0x41
CPU architecture: 8
CPU variant	: 0x0
CPU part	: 0xd05
CPU revision	: 1

processor	: 1
BogoMIPS	: 52.00
Features	: fp asimd evtstrm aes pmull sha1 sha2 crc32
CPU implementer	: 0x41
CPU architecture: 8
CPU variant	: 0x0
CPU part	: 0xd05
CPU revision	: 1

processor	: 2
BogoMIPS	: 52.00
Features	: fp asimd evtstrm aes pmull sha1 sha2 crc32
CPU implementer	: 0x41
CPU architecture: 8
CPU variant	: 0x0
CPU part	: 0xd05
CPU revision	: 1

processor	: 3
BogoMIPS	: 52.00
Features	: fp asimd evtstrm aes pmull sha1 sha2 crc32
CPU implementer	: 0x41
CPU architecture: 8
CPU variant	: 0x0
CPU part	: 0xd05
CPU revision	: 1

processor	: 4
BogoMIPS	: 52.00
Features	: fp asimd evtstrm aes pmull sha1 sha2 crc32
CPU implementer	: 0x53
CPU architecture: 8
CPU variant	: 0x1
CPU part	: 0x002
CPU revision	: 0

processor	: 5
BogoMIPS	: 52.00
Features	: fp asimd evtstrm aes pmull sha1 sha2 crc32
CPU implem

In [28]:
reg = LinearRegression(n_jobs=8)
reg.fit(x, y)

LinearRegression(n_jobs=8)

### Find the intercept

In [29]:
reg.intercept_

-5772267.017463278

### Find the coefficients

In [30]:
reg.coef_

array([ 227.70085401, 2916.78532684])

### Calculate the R-squared

In [31]:
reg.score(x, y)

0.7764803683276793

### Calculate the Adjusted R-squared

In [32]:
x.shape

(100, 2)

In [33]:
def get_adj_r2(x, y):
    n = x.shape[0]
    p = x.shape[1]
    r2adj = 1 - (1 - r2) * ((n - 1) / (n - p - 1))
    return r2adj


get_adj_r2(x, y)

0.77187171612825

### Compare the R-squared and the Adjusted R-squared

Answer: R-squared is bigger than the Adjusted R-squared (as expected) but difference is not much.

### Compare the Adjusted R-squared with the R-squared of the simple linear regression

Answer:
R^2 of Simple Regression (size only) = 0.7447391865847574
Adjusted R^2 (size and year) = 0.77187171612825

It increased when we added another feature. But, the jumpt is not much.

### Making predictions

Find the predicted price of an apartment that has a size of 750 sq.ft. from 2009.

In [34]:
new_data = pd.DataFrame({"size": [750]})
new_data["year"] = [2009]
reg.predict(new_data)

array([258330.34465995])

### Calculate the univariate p-values of the variables

In [40]:
# The difference is that we can check what's contained in the local variable 'p' in an instance of the LinearRegression() class
reg.p

array([0., 0.])

In [41]:
from sklearn.feature_selection import f_regression

In [42]:
f_regression(x, y)

(array([285.92105192,   0.85525799]), array([8.12763222e-31, 3.57340758e-01]))

In [43]:
p_values = f_regression(x, y)[1]
p_values

array([8.12763222e-31, 3.57340758e-01])

In [44]:
p_values.round(3)

array([0.   , 0.357])

### Create a summary table with your findings

In [46]:
# Let's create a new data frame with the names of the features
reg_summary = pd.DataFrame([["size"], ["year"]], columns=["Features"])
# Then we create and fill a second column, called 'Coefficients' with the coefficients of the regression
reg_summary["Coefficients"] = reg.coef_
# Finally, we add the p-values we just calculated
reg_summary["p-values"] = p_values.round(3)
reg_summary

,Features,Coefficients,p-values
0,size,227.700854,0.000
1,year,2916.785327,0.357


Answer: Looking a at the p-values (via f_regression), adding year is not significant.